In [96]:
#Imports
import pandas as pd
import matplotlib as mpl
from mplsoccer.pitch import VerticalPitch, Pitch
from matplotlib.colors import to_rgba
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colorbar
from matplotlib.colors import LinearSegmentedColormap
from highlight_text import fig_text
import ipywidgets as widgets
import math
from scipy.spatial import ConvexHull
from scipy import stats

#Remove warning
pd.options.mode.chained_assignment = None

#Read Data
df = pd.read_csv('Footballdata/MS/SogdianavsKL.csv')
Eventdf = pd.read_csv('OptaEventName.csv')

#Merge to obtain accurate position of player
df = pd.merge(df, Eventdf, on="typeId").sort_values('timeStamp', ascending = True).reset_index()

#Get Headers, endX and endY
df['endX'] = 0.0
df['endY'] = 0.0
df['head'] = 0
df['Cross'] = 0
df['SetPiece'] = 0
df['FreeKick'] = 0
df['CornerKick'] = 0
df['ThrowIn'] = 0
df['Error'] = 0
df['Length'] = 0.0
df['BlockedX'] = 0.0
df['BlockedY'] = 0.0
df['GoalMouthY'] = 0.0
df['GoalMouthZ'] = 0.0
df['GoalMouthZ'] = 0.0

type_cols = [col for col in df.columns if '/qualifierId' in col]
for i in range(len(df)):
    df1 = df.iloc[i:i+1,:]
    x = 0 
    for x in range(len(type_cols)):
        col = df1[type_cols[x]].values[0]
        if col == 140:
            endx = df1.loc[:,'qualifier/%i/value' %x].values[0]
            df['endX'][i] = endx
        else:
            x +=1
    y = 0
    for y in range(len(type_cols)):
        col = df1[type_cols[y]].values[0]
        if col == 141:
            endy = df1.loc[:,'qualifier/%i/value' %y].values[0]
            df['endY'][i] = endy
        else:
            y +=1
    h = 0
    for h in range(len(type_cols)):
        col = df1[type_cols[h]].values[0]
        if col == 15:
            head = df1.loc[:,'qualifier/%i/value' %h].values[0]
            df['head'][i] = 1
        else:
            h +=1  
    c = 0
    for c in range(len(type_cols)):
        col = df1[type_cols[c]].values[0]
        if col == 2:
            cross = df1.loc[:,'qualifier/%i/value' %c].values[0]
            df['Cross'][i] = 1
        else:
            c +=1          
    fk = 0
    for fk in range(len(type_cols)):
        col = df1[type_cols[fk]].values[0]
        if col == 5:
            fk = df1.loc[:,'qualifier/%i/value' %fk].values[0]
            df['FreeKick'][i] = 1
        else:
            fk +=1  
    ck = 0
    for ck in range(len(type_cols)):
        col = df1[type_cols[ck]].values[0]
        if col == 6:
            ck = df1.loc[:,'qualifier/%i/value' %ck].values[0]
            df['CornerKick'][i] = 1
        else:
            ck +=1  
    ti = 0
    for ti in range(len(type_cols)):
        col = df1[type_cols[ti]].values[0]
        if col == 107:
            ti = df1.loc[:,'qualifier/%i/value' %ti].values[0]
            df['ThrowIn'][i] = 1
        else:
            ti +=1
    e = 0
    for e in range(len(type_cols)):
        col = df1[type_cols[e]].values[0]
        if col == 169:
            e = df1.loc[:,'qualifier/%i/value' %e].values[0]
            df['Error'][i] = 1
        else:
            e +=1
    l = 0
    for l in range(len(type_cols)):
        col = df1[type_cols[l]].values[0]
        if col == 212:
            length = df1.loc[:,'qualifier/%i/value' %l].values[0]
            df['Length'][i] = length
        else:
            l +=1
    sp = 0
    for sp in range(len(type_cols)):
        col = df1[type_cols[sp]].values[0]
        if ((col == 5) | (col == 6) | (col == 107)):
            sp = df1.loc[:,'qualifier/%i/value' %sp].values[0]
            df['SetPiece'][i] = 1
        else:
            sp +=1
    bx = 0 
    for bx in range(len(type_cols)):
        col = df1[type_cols[bx]].values[0]
        if col == 146:
            blockx = df1.loc[:,'qualifier/%i/value' %bx].values[0]
            df['BlockedX'][i] = blockx
        else:
            bx +=1
    by = 0 
    for by in range(len(type_cols)):
        col = df1[type_cols[by]].values[0]
        if col == 147:
            blocky = df1.loc[:,'qualifier/%i/value' %by].values[0]
            df['BlockedY'][i] = blocky
        else:
            by +=1
    gy = 0
    for gy in range(len(type_cols)):
        col = df1[type_cols[gy]].values[0]
        if col == 102:
            goaly = df1.loc[:,'qualifier/%i/value' %gy].values[0]
            df['GoalMouthY'][i] = goaly
        else:
            gy +=1            
    gz = 0
    for gz in range(len(type_cols)):
        col = df1[type_cols[gz]].values[0]
        if col == 103:
            goalz = df1.loc[:,'qualifier/%i/value' %gz].values[0]
            df['GoalMouthZ'][i] = goalz
        else:
            gz +=1             
            
#Adjust to pitch
df['x'] = df['x']*1.2
df['y'] = df['y']*.8
df['endX'] = df['endX']*1.2
df['endY'] = df['endY']*.8
df['BlockedX'] = df['BlockedX']*1.2
df['BlockedY'] = df['BlockedY']*.8

#Make a new, single column for time and sort the events in chronological order
df["Time"] = (60*df["timeMin"]) + df["timeSec"]

#New Column for Headers
df['Headers'] = np.where(((df['head'] == 1) | ((df['typeName'] == 'Aerial') & (df['outcome'] == 1))), 1, 0)

#New Column for Assists
df['Assist']= np.where((df['keyPass'] == 1) & (df['typeName'].shift(-1) == 'Goal'),1,0)

#New Column for Assists
df['Goal']= np.where(df['typeName'] == 'Goal',1,0)

#Change Key Passes to 0
df['keyPass'] = df['keyPass'].fillna(0).astype(int)

#Change Key Passes to 0
df['Number'] = df['Number'].fillna(0).astype(int)

#relevant variables
df= df[['id','Time','playerName','Number','contestantId','typeName','x','y','endX','endY','Length',"FreeKick",
        "CornerKick","ThrowIn",'SetPiece','Headers','Cross','keyPass','Assist','Error','BlockedX',
        'BlockedY', 'GoalMouthY', 'GoalMouthZ','Goal','outcome']]

df.rename(columns = {'playerName':'Player', 'contestantId':'Team', 'id':'ID',
                     'typeName':'Event', 'keyPass':'KeyPass', 'outcome':'Outcome'}, inplace = True)

#make a copy
df2 = df

#df['Event'].unique()

In [97]:
#Retrieve Team
#team = teamlist.value
team = 'Kuala Lumpur City FC'

#Retrieve Opposition team (only use when there's two available teams for selection)
oppo = df.loc[(df['Team']!= team)]
oppo = oppo['Team'].unique()
oppo = oppo[0]

df.sort_values(by=['Time'])

#find time of the team's first substitution and filter the df to only passes before that
Subs = df.loc[(df['Event']== 'Player off') & (df['Team'] == team)]
SubTimes = Subs["Time"]
SubOne = SubTimes.min()

#remove substitution events,challenges & formation change
df = df.loc[~(df['Event'] == 'Team set up')]
df = df.loc[~(df['Event'] == 'Corner Awarded')]
df = df.loc[~(df['Event'] == 'Formation change')]
df = df.loc[~(df['Event'] == 'Challenge')]
df = df.loc[~(df['Event'] == 'Shield ball opp')]
df = df.loc[~(df['Event'] == 'Start')]
df = df.loc[~(df['Event'] == 'End')]
df = df.loc[~(df['Event'] == 'Out')]
df = df.loc[~(df['Event'] == 'Player off')]
df = df.loc[~(df['Event'] == 'Player on')]
df = df.loc[~(df['Event'] == 'Card')]
df = df.loc[~(df['Event'] == 'Deleted event')]
df = df.loc[~(df['Event'] == 'Offside Pass')]
df = df.loc[~(df['Event'] == 'Offside provoked')]
df = df.loc[~(df['Event'] == 'Start delay')]
df = df.loc[~(df['Event'] == 'End delay')]
df = df.loc[~(df['Event'] == 'Collection End')]

#remove the records of a failed aerial, they should be ignored as their challenges are failed and the shot continues
df = df.loc[~((df['Event'] == 'Aerial') & (df['Outcome'] == 0))]

#remove the records of a failed aerial, they should be ignored as their challenges are failed and the shot continues
df = df.loc[~((df['Event'] == 'Tackle') & (df['Outcome'] == 0))].reset_index(drop=True)

#New Column Next & Prev Event
df['Prev'] = df['Event'].shift(1)
df['Next'] = df['Event'].shift(-1)
df['NextT'] = df['Team'].shift(-1)

In [98]:
#Remove warning
pd.options.mode.chained_assignment = None

#filter for only actions by the team you want
OneTeam = df.loc[(df['Team']== team)]

#identify the passer and then the recipient, who'll be the playerId of the next action
OneTeam['passer'] = OneTeam['Number']
OneTeam['recipient'] = OneTeam['passer'].shift(-1)

#Filter to make sure recipients are from the same team
OneTeam = OneTeam.loc[(OneTeam['NextT'] == OneTeam['Team'])]

#filter for only passes and then successful passes
Passes = OneTeam.loc[(OneTeam['Event']=="Pass") & (OneTeam['SetPiece']== 0)]
Completions_df = Passes.loc[(Passes['Outcome']== 1)]

#Downcast Kit numbers from float to integer
pas = pd.to_numeric(Completions_df['passer'],downcast='integer')
rec = pd.to_numeric(Completions_df['recipient'],downcast='integer')

#New columns for passer and recipient
Completions_df['passer'] = pas
Completions_df['recipient'] = rec

#Set Completions before first sub is made // Completions is df that is filtered by first sub; Completions_df is not
#Completions = Completions_df.loc[Completions_df['period/displayName'] == 'SecondHalf']
#Completions = Completions.loc[Completions['newsecond'] > 2700]
Completions = Completions_df.loc[Completions_df['Time'] < SubOne]

#Obtaining passing combinations (for full game, so use Completions_df because it includes full game)
pass_combo = Completions_df.groupby(['passer', 'recipient']).ID.count().reset_index()
pass_combo.columns = ['Passer', 'Recipient', 'Passing Combinations']
pass_combo = pass_combo.sort_values(by=['Passing Combinations'] , ascending=False)

In [99]:
#Find Average Locations with Completions, EPV is filtered till first sub
average_locs_and_count = Completions.groupby('passer').agg({'x': ['median'], 'y': ['median','count']})
average_locs_and_count.columns = ['x', 'y', 'count']
average_locs_and_count

#Size of the node later on, dependent on the passes made
player_pass_count = Completions.groupby("passer").size().to_frame("num_passes")

#Find out the passes made till first sub is made
player_pass_count1 = player_pass_count
#average_locs_and_count = average_locs_and_count.drop(17)
average_locs_and_count

,x,y,count
passer,,,
1,16.92,39.04,10
4,27.30,17.88,12
6,46.68,31.52,15
7,88.56,67.52,9
8,67.92,14.32,14
9,29.88,32.00,7
12,41.46,72.88,8
14,44.52,44.08,21
16,75.60,69.92,11


In [100]:
#find number of passes along each 'path' of passer to recipient
passes_between = Completions.groupby(['passer', 'recipient']).ID.count().reset_index()
passes_between.rename({'ID': 'pass_count'}, axis='columns', inplace=True)

passes_between = passes_between.merge(average_locs_and_count, left_on='passer', right_index=True)
passes_between = passes_between.merge(average_locs_and_count, left_on='recipient', right_index=True,
                                      suffixes=['', '_end'])

#set minimum threshold for pass arrows to be plotted. So this will only plot combos which occured at least 5 times.
number = 2
passes_between_df = passes_between.loc[(passes_between['pass_count']>number)]
number=str(number)

#Line width
line = passes_between_df.pass_count / passes_between_df.pass_count.max()
#passes_between_df

In [102]:
#Set up pitch
mpl.rcParams['figure.dpi'] = 500
pitch = VerticalPitch(pitch_type='statsbomb', pitch_color='#000000', line_color='white',
                      half = False, line_zorder=1, linewidth=5, spot_scale=0.006)

fig,ax = pitch.draw(figsize=(30, 20), constrained_layout=False, tight_layout=True)
fig.set_facecolor('#000000')

#Inverse the y-axis
plt.gca().invert_xaxis()

pitch.lines(passes_between_df.x, passes_between_df.y, passes_between_df.x_end,passes_between_df.y_end, 
            linewidth=line*30, color='yellow', ax=ax, zorder = 3, alpha = .7,ls='--')

#plot nodes
pass_nodes = pitch.scatter(average_locs_and_count.x, average_locs_and_count.y, s=7000, marker='h',
                           color='#E40019', edgecolors="w", linewidth=6, alpha=1, ax=ax, zorder=3)

#Kit Numbers on Nodes
for index, row in average_locs_and_count.iterrows():
    pitch.annotate(row.name, xy=(row.x-0.25,row.y), c='w', va='center', ha='center', 
                   size=35, fontweight='bold', fontfamily="Courier New",ax=ax)

#Title
highlight_textprops =\
    [{"color": "#E40019","fontweight": 'bold'}]

fig_text(s = "\n<KL City's> Passing Network",
        x = 0.5, y = 1.08, highlight_textprops = highlight_textprops ,color='white',
        fontfamily = "Courier New",fontweight='bold', ha='center',va='center',fontsize=60)

highlight_textprops =\
    [{"color": "#1d9bf0","fontweight": 'bold'}]

fig_text(s = "vs <FC Sogdiana> | AFC Cup Semi Final",
        x = 0.5, y = 1.035, highlight_textprops = highlight_textprops ,color='white',
        fontfamily = "Courier New",fontweight='bold', ha='center',va='center',fontsize=50)

highlight_textprops =\
    [{"color": "yellow","fontweight": 'bold'}]

fig_text(s = "<Line Width> = Frequency of Combination" ,
        x = 0.5, y = 1.005, highlight_textprops = highlight_textprops ,color='white',
        fontfamily = "Courier New",fontweight='bold', ha='center',fontsize=35)

#annotations/key at bottom
pitch.annotate("Connections With Over " + number + " Passes Included, Before First Sub", 
               (-2.25, 40), color='w', fontsize=20, ha='center', va='center', ax=ax, 
               fontweight='bold', fontfamily="Courier New")

#Space
ax.text(95, -4.5, 'space', va='center', ha='left', fontsize=25, color='black',
        fontweight = "bold",fontfamily = "Courier New")

ax.text(-8, -4.5, 'space', va='center', ha='left', fontsize=25, color='black',
        fontweight = "bold",fontfamily = "Courier New")

fig_text(s = "Space",x = 0.5, y = 1.125 ,color='black',
         fontfamily = "Courier New",fontweight='bold',
         ha='center',va='center',fontsize=10)

#credits
fig.text(.7,.06,'@chunhang7\nData via Opta',fontsize=20, fontfamily='Courier New', 
         ha='right',color='w',fontweight='bold')

plt.show()